<a href="https://colab.research.google.com/github/arlestotle/home-sales-challenge/blob/main/Home_Sales_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.2'
spark_version = 'spark-3.4.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (129 kB/s)
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

# Show the dataframe
home_sales.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
home_sales.createOrReplaceTempView('home_sales')


In [8]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

avg_price_4br = """
SELECT
  YEAR(date) AS year,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY year
ORDER BY year DESC
"""
spark.sql(avg_price_4br).show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [9]:
# 4. What is the average price of a home for each year the home was built, that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

avg_price_3br_3bath = """
SELECT
  YEAR(date_built) AS year_built,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3
  and bathrooms = 3
GROUP BY year_built
ORDER BY year_built DESC
"""
spark.sql(avg_price_3br_3bath).show()



+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2017|    292676.79|
|      2016|    290555.07|
|      2015|     288770.3|
|      2014|    290852.27|
|      2013|    295962.27|
|      2012|    293683.19|
|      2011|    291117.47|
|      2010|    292859.62|
+----------+-------------+



In [10]:
# 5. What is the average price of a home for each year the home was built, that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

avg_price_3br_3bath_2f_sf2000 = """
SELECT
  YEAR(date_built) AS year_built,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3
  and bathrooms = 3
  and floors = 2
  and sqft_living >= 2000
GROUP BY year_built
ORDER BY year_built DESC
"""
spark.sql(avg_price_3br_3bath_2f_sf2000).show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [11]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

avg_price_view_350000 = """
SELECT
  view,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(avg_price_view_350000).show()
## HAVING clause in SQL specidied that an SQL SELECT statement must only return rows were aggregate values meet the specified conditions

## From Colab AI, sorts the view by first first digit?
# avg_price_view_350000 = """
# SELECT
#   view,
#   ROUND(AVG(price), 2) AS average_price
# FROM (
#   SELECT *
#   FROM home_sales
#   WHERE price >= 350000
# ) AS filtered_sales
# GROUP BY view
# ORDER BY view DESC
# """
# spark.sql(avg_price_view_350000).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.591944694519043 seconds ---


In [12]:
# 7. Cache the the temporary table home_sales.
# Cashe tables to store data that you access frequently but that does not change often.
# A cache table can improve  performance by storing the data locally instead of accessing the data directly from the data source. (From google)
spark.sql("cache table home_sales")


DataFrame[]

In [13]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [14]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
start_time = time.time()

avg_price_view_350000_cached = """
SELECT
  view,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(avg_price_view_350000_cached).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.5867002010345459 seconds ---


In [15]:
# Determine the runtime and compare it to the uncached runtime.
print("Using the cached data the runtime was 0.586 seconds whereas the uncached data runtime was 1.591. \n Cached data is used to quickly load the application or website's information every time the user subsequently opens or visits it.")

Using the cached data the runtime was 0.586 seconds whereas the uncached data runtime was 1.591. 
 Cached data is used to quickly load the application or website's information every time the user subsequently opens or visits it.


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales.write.partitionBy('date_built').parquet('parquet_home_sales', mode = 'overwrite')

In [18]:
# 11. Read the parquet formatted data.
parquet_home_sales_df = spark.read.parquet('parquet_home_sales')

In [19]:
# 12. Create a temporary table for the parquet data.
home_sales.createOrReplaceTempView('parquet_home_sales')


In [20]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
start_time = time.time()

avg_price_view_350000_parquet = """
SELECT
  view,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(avg_price_view_350000_parquet).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.5624356269836426 seconds ---


In [21]:
# Determine the runtime and compare it to the cached runtime.
print("Using the parquet data the runtime was 0.562 seconds. \n Using the cached data the runtime was 0.586 seconds. \n The parquet files load slightly faster than the cached data because the parquet data is organized into row groups, which are further divided into pages. \n This allows for fast data access, as only the necessary row groups and pages need to be loaded into memory.")

Using the parquet data the runtime was 0.562 seconds. 
 Using the cached data the runtime was 0.586 seconds. 
 The parquet files load slightly faster than the cached data because the parquet data is organized into row groups, which are further divided into pages. 
 This allows for fast data access, as only the necessary row groups and pages need to be loaded into memory.


In [22]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [23]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False